In [1]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import numpy as np

In [2]:
df = yf.download('spy', start='2020-01-01')

[*********************100%%**********************]  1 of 1 completed


In [3]:
# wave trend

# variables basis cipher B settings

n1 = 9 #channel length
n2 = 12 #average length

df['ap'] = (df.High+df.Low+df.Close)/3 #WT ma source
df['esa'] = ta.ema(df.ap,n1) #EMA of the ma source by the channel length
df['d'] = ta.ema(abs(df.ap-df.esa),n1)
df['ci'] = (df.ap-df.esa)/(0.015*df.d)
df['wt1'] = ta.ema(df.ci, n2)
df['wt2'] = ta.sma(df.wt1,3)

In [4]:
# MF RSI

# variables

period = 60
multi = 150
posy = 2.5

df['mfrsi'] = (ta.sma(((df.Close-df.Open)/(df.High-df.Low)*multi),period))-posy

In [5]:
# moving averages

df['ma50'] = ta.ema(df.Close, 50)
df['ma200'] = ta.ema(df.Close, 200)

**Rules**

1) can only go long when price is above the 200 EMA and can only take shorts when price is below the 200 EMA

2) Can only take a position when price has pulled back into the 50 EMA

3) Can only go long if mf is positive and short only when mf is negative

4) Can only go long when wave trend is negative and only go short when wave trend is positive

entry on close via a wave trend cross stop below the swing low/ above the swing high, profit at 2x the stop

In [6]:
# stop points

df['HH'] = df.High.rolling(5).max()
df['LL'] = df.Low.rolling(5).min()

In [7]:
columns_to_drop = ['Adj Close', 'Volume', 'ap', 'esa', 'd', 'ci']
df.drop(columns=columns_to_drop, inplace=True)

In [8]:
df = round(df,2)

In [9]:
df = df.dropna()

In [10]:
df['cross'] = np.select([(df['wt1'].shift(1) < df['wt2'].shift(1)) & (df['wt1'] > df['wt2']),
                         (df['wt1'].shift(1) > df['wt2'].shift(1)) & (df['wt1'] < df['wt2'])],
                        ['up','down'])

In [12]:
df['signal'] = np.select([(df.cross=='up')&
                          (df.Close>df.ma200)&
                          (df.LL<df.ma50)&
                          (df.mfrsi>0)&
                          (df.wt1<0)&
                          (df.wt2<0),
                          (df.cross=='down')&
                          (df.Close<df.ma200)&
                          (df.HH>df.ma50)&
                          (df.mfrsi<0)&
                          (df.wt1>0)&
                          (df.wt2>0)],
                         [1,2])
                          

In [13]:
df

,Open,High,Low,Close,wt1,wt2,mfrsi,ma50,ma200,HH,LL,cross,signal
Date,,,,,,,,,,,,,
2020-10-15,343.71,348.02,343.13,347.50,53.62,58.41,18.29,336.15,311.69,354.02,343.13,0,0
2020-10-16,348.96,350.75,347.10,347.29,51.84,55.17,18.68,336.58,312.04,354.02,343.13,0,0
2020-10-19,348.65,349.33,341.04,342.01,40.69,48.72,16.74,336.80,312.34,352.47,341.04,0,0
2020-10-20,343.46,346.88,342.64,343.38,31.86,41.46,15.18,337.06,312.65,351.93,341.04,0,0
2020-10-21,343.33,345.67,342.40,342.73,22.54,31.69,15.86,337.28,312.95,350.75,341.04,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-01,419.20,423.50,418.65,422.66,-42.17,-52.47,-12.75,430.53,425.49,423.50,409.21,0,0
2023-11-02,426.58,430.92,426.56,430.76,-20.50,-39.23,-8.42,430.54,425.54,430.92,409.21,0,0
2023-11-03,433.14,436.29,433.01,434.69,-0.39,-21.02,-6.42,430.70,425.63,436.29,412.22,0,0


In [15]:
trades = []
entry = 0
exit = 0
stop = 0
profit = 0
entry_date = 0
exit_date = 0
position = None

for i in range(len(df)):
    if position == None:
        
        # long entry
        if df.signal.iloc[i] == 1:
            entry = df.Close.iloc[i]
            stop = df.LL.iloc[i]
            profit = (entry - stop) * 2 + entry
            entry_date = df.index[i]
            position = 'Long'
        
        # short entry
        elif df.signal.iloc[i] == 2:
            entry = df.Close.iloc[i]
            stop = df.HH.iloc[i]
            profit = entry - (stop - entry) * 2
            entry_date = df.index[i]
            position = 'Short'
            
     # long exit
    if position == 'Long':
        if df.High.iloc[i] > profit:
            exit = profit
            exit_date = df.index[i]
        elif df.Low.iloc[i] <= stop:
            exit = stop
            exit_date = df.index[i]
            
    # short exit
    if position == 'Short':
        if df.Low.iloc[i] < profit:
            exit = profit
            exit_date = df.index[i]
        elif df.High.iloc[i] >= stop:
            exit = stop
            exit_date = df.index[i]   
     
    # record the trades
    if exit !=0:
        if position == 'Long':
            trades.append({'Entry Date':entry_date,
                         'Direction': position,
                         'Entry Price': entry,
                         'Profit': profit,
                         'Stop': stop,
                         'Exit Price': exit,
                         'Exit Date':exit_date,
                         'PL':exit - entry})
            
        elif position == 'Short':
            trades.append({'Entry Date':entry_date,
                         'Direction': position,
                         'Entry Price': entry,
                         'Profit': profit,
                         'Stop': stop,
                         'Exit Price': exit,
                         'Exit Date':exit_date,
                         'PL':entry - exit})
            
        entry_date = 0
        position = None
        entry = 0
        profit = 0
        stop = 0
        exit = 0
        exit_date = 0
        
        
trades_df = pd.DataFrame(trades)       
            

In [16]:
trades_df

,Entry Date,Direction,Entry Price,Profit,Stop,Exit Price,Exit Date,PL
0,2020-11-03,Long,336.03,362.89,322.60,362.89,2020-11-09,26.86
1,2021-03-02,Long,386.54,403.16,378.23,378.23,2021-03-04,-8.31
2,2021-05-14,Long,416.58,441.74,404.00,441.74,2021-07-29,25.16
3,2021-09-23,Long,443.18,471.82,428.86,428.86,2021-09-30,-14.32
4,2021-10-05,Long,433.10,446.58,426.36,446.58,2021-10-18,13.48
5,2021-12-06,Long,458.79,478.53,448.92,478.53,2021-12-28,19.74
6,2022-01-12,Long,471.02,499.86,456.60,456.60,2022-01-18,-14.42
7,2022-04-19,Long,445.04,463.90,435.61,435.61,2022-04-22,-9.43
8,2022-11-02,Short,374.87,343.83,390.39,390.39,2022-11-10,-15.52
9,2022-11-17,Short,394.24,378.10,402.31,402.31,2022-11-23,-8.07
